# Dimensional Lumber Scraper 

One of the most startling side-effects of the Covid-19 pandemic has been the sharp increase in building materials. **Dimensional lumber (standard '2x4's and the like) saw increases of over 400% in some regions.** For fellow small business operators in the carpentry / cabinetmaking space it is essential to understand and monitor the volatility of these fundamental materials. 

The objective of this workbook is to create a web scraper that gathers Dimensional Lumber product data from major retailers Home Depot and Lowes. The end product is a merged csv file, timestamped by date. A follow-up project to this will utilize these outputs to provide immediate price comparisons and analyze price volatility over time from a local retail level.

This workbook is separated into 3 primary sections: 
1. **Home Depot** - we scrape Home Depot's information, clean it, and format in a DataFrame.
2. **Lowes** - we repeat the same process as Home Depot, altering our code to account for differences in the data. 
3. **Merge & Ouput** - we merge each sources data and output to a csv file. 


In [1]:
#Import all necessary packages

import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import datetime as dt
from selenium.webdriver import Chrome
import os

## Home Depot

For Home Depot we will be scraping the following url: 

https://www.homedepot.ca/en/home/categories/building-materials/lumber-and-composites/dimensional-lumber-and-studs.html

In [2]:
# If this block is slow to run, change header user-agent to your browser's current user-agent. 

hd_url = 'https://www.homedepot.ca/en/home/categories/building-materials/lumber-and-composites/dimensional-lumber-and-studs.html'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
request = requests.get(hd_url, headers = header)
content = request.content
soup = BeautifulSoup(content, 'html.parser')


In [3]:
#After quickly looking at the soup output, it was identified that the fourth'<script> tag contained our target data in json format'

target_content = soup.find_all('script')[3]
target_string = target_content.string
target_json = json.loads(target_string)

# Trial & error, previously explored which dictionary keys were required to access wanted content
target_iterable = target_json['mainEntity']['offers']['itemOffered']

# Iterate through Json, adding list of wanted variables to 'rows' list, create DataFrame with Rows list once complete
rows = []
first_layer = ['sku','name','description','url','image']

for iter_list in target_iterable: 
    temp_list = []
    for layer in first_layer:
        temp_list.append(iter_list[layer])
    
    #Price isn't always available, check to see if parent key exists, grab value if it does. 
    if iter_list.get('offers'):
        temp_list.append(iter_list['offers']['price'])
    else: 
        temp_list.append(None)
    
    # Avg_rating and review_count have a parent key
    temp_list.append(iter_list['aggregateRating']['ratingValue'])
    temp_list.append(iter_list['aggregateRating']['reviewCount'])
    
    #Add timestamp for record
    temp_list.append(dt.date.today())
    
    rows.append(temp_list)

    
#Create dataframe 
cols = ['sku', 'name','description','url','image', 'price', 'avg_rating', 'review_count','date']
hd_df = pd.DataFrame(rows, columns = cols)

hd_df.head()

,sku,name,description,url,image,price,avg_rating,review_count,date
0,1000112108,2-inch x 4-inch x 8 ft. SPF Dimensional Lumber,2-inch x 4-inch x 8 ft. SPF Dimensional Lumber,https://www.homedepot.ca/product/2-inch-x-4-in...,https://homedepot.scene7.com/is/image/homedepo...,6.58,3.7443,884,2021-09-24
1,1000100158,2x4x10 SPF Dimension Lumber,2x4x10 SPF Dimension Lumber,https://www.homedepot.ca/product/2x4x10-spf-di...,https://homedepot.scene7.com/is/image/homedepo...,13.48,3.8594,192,2021-09-24
2,1000173739,1x2x8 Framing Lumber,HDG 1x2x8 Framing Lumber,https://www.homedepot.ca/product/hdg-1x2x8-fra...,https://homedepot.scene7.com/is/image/homedepo...,3.78,3.7595,158,2021-09-24
3,1000117995,2x6x8 SPF Dimension Lumber,Orbit 2x6x8 SPF Dimension Lumber,https://www.homedepot.ca/product/orbit-2x6x8-s...,https://homedepot.scene7.com/is/image/homedepo...,14.33,3.9756,369,2021-09-24
4,1000414127,1- Inch X 6-inch X 8 Ft. Spf Kd,Millstead 1- Inch X 6-inch X 8 Ft. Spf Kd,https://www.homedepot.ca/product/millstead-1-i...,https://homedepot.scene7.com/is/image/homedepo...,NaN,2.8925,93,2021-09-24


#### Cleaning Home Depot Columns: 

Let's clean some of our columns to make them easier handle and extract some important information embedded within some columns. 
- Extract dimension data, create separate columns for thickness, width, and length
- Transform to appropriate data types, rearrange columns etc.  

In [4]:
# Create function to extract dimensions from description, searching for format matches with regex 
# Common convention for lumber naming is that product thickness, width, and length appear in that order

def get_dimensions(cell):
    matches = re.findall(r'\d+\.?\d*', cell)
    thickness, width = float(matches[0]), float(matches[1])
    
    # The Length dimension, conventionally the third value in the product description, isn't always specified depending on the product
    # Adjust function to add null "Length" value case. 
       
    if len(matches) > 2:
        length = float(matches[2])
    else: 
        length = None
    
    return thickness, width, length

hd_df['dimensions'] = hd_df['description'].apply(get_dimensions)
hd_df[['thickness','width','length']] = hd_df['dimensions'].apply(pd.Series)

#Reorder columns, drop 'name' column - same info is present in 'description' column, add 'retailer' column, adjust data types 

hd_df.drop(columns = 'name', inplace = True)
hd_df['retailer'] = 'Home Depot'
hd_df['sku'] = hd_df['sku'].astype('int')
hd_df['date'] = pd.to_datetime(hd_df['date'])

reorder_cols = ['sku', 'description','retailer','price', 'dimensions', 'thickness', 'width', 'length','avg_rating', 'review_count', 'date', 'url', 'image'] 
hd_df = hd_df[reorder_cols]


hd_df.head(3)


,sku,description,retailer,price,dimensions,thickness,width,length,avg_rating,review_count,date,url,image
0,1000112108,2-inch x 4-inch x 8 ft. SPF Dimensional Lumber,Home Depot,6.58,"(2.0, 4.0, 8.0)",2.0,4.0,8.0,3.7443,884,2021-09-24,https://www.homedepot.ca/product/2-inch-x-4-in...,https://homedepot.scene7.com/is/image/homedepo...
1,1000100158,2x4x10 SPF Dimension Lumber,Home Depot,13.48,"(2.0, 4.0, 10.0)",2.0,4.0,10.0,3.8594,192,2021-09-24,https://www.homedepot.ca/product/2x4x10-spf-di...,https://homedepot.scene7.com/is/image/homedepo...
2,1000173739,HDG 1x2x8 Framing Lumber,Home Depot,3.78,"(1.0, 2.0, 8.0)",1.0,2.0,8.0,3.7595,158,2021-09-24,https://www.homedepot.ca/product/hdg-1x2x8-fra...,https://homedepot.scene7.com/is/image/homedepo...


In [5]:
hd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   sku           20 non-null     int64         
 1   description   20 non-null     object        
 2   retailer      20 non-null     object        
 3   price         16 non-null     float64       
 4   dimensions    20 non-null     object        
 5   thickness     20 non-null     float64       
 6   width         20 non-null     float64       
 7   length        20 non-null     float64       
 8   avg_rating    20 non-null     float64       
 9   review_count  20 non-null     int64         
 10  date          20 non-null     datetime64[ns]
 11  url           20 non-null     object        
 12  image         20 non-null     object        
dtypes: datetime64[ns](1), float64(5), int64(2), object(5)
memory usage: 2.2+ KB


## Repeat Process for Lowes

Now that we have our Home Depot data cleaned and available in a single dataframe, let's repeat the process for Lowes. 

Since the Lowes data is loaded dynamically a few seconds after the intitial page, we need to use the Selenium package to obtain the data. The webpage we will be scraping is the following: 

https://www.lowes.ca/dept/dimensional-lumber-lumber-building-supplies-a541?display=24


In [6]:
lowes_url = 'https://www.lowes.ca/dept/dimensional-lumber-lumber-building-supplies-a541?display=24'

#Ensure Chromedriver is downloaded & located in current working directory 
driver = Chrome(executable_path = os.getcwd()+'/chromedriver')
driver.get(lowes_url)
page = driver.page_source
driver.quit()

soup = BeautifulSoup(page, 'html.parser')

In [14]:
# Lowes data is all contained in html tags. 
# Loop through each product and parse out the needed info (tags / hierarchy uncovered via trial & error looking at page source code)

items = []

for item in soup.find_all('div', class_='card-product'):
    
    image = item.img.get('src')
    price = item.find('div', class_='price-actual').text
    sku = item.get('data-productid')
    description = item.find('a', class_="link-black d-block").text
    avg_rating = item.find_all('a', class_="smooth-scroll trigger-drawer expand-product-reviews")[0].get('aria-label')
    review_count = item.find_all('a', class_="smooth-scroll trigger-drawer expand-product-reviews")[1].get('aria-label')
    date = dt.date.today()

    items.append([sku, description, price, avg_rating, review_count, image, date])

    
columns = ['sku', 'description', 'price', 'avg_rating', 'review_count', 'image', 'date']
lowes_df = pd.DataFrame(items, columns = columns)

lowes_df.head()

,sku,description,price,avg_rating,review_count,image,date
0,49127,2-in x 12-in x 8-ft Kiln-Dried Whitewood Lumber,$24.89,1 out of 5 starts. Read reviews.,1. Read reviews.,https://da.lowes.ca/webassets/images/49127_019...,2021-09-24
1,28512,Canfor 2-in x 4-in x 14-ft Spruce Pine Fir Lum...,$10.99,4.37 out of 5 starts. Read reviews.,89. Read reviews.,https://da.lowes.ca/webassets/images/28512_089...,2021-09-24
2,330614202,2-in x 12-in Kiln-Dried SPF Lumber,$31.49 to $50.38,1 out of 5 starts. Read reviews.,1. Read reviews.,https://da.lowes.ca/webassets/images/49227_019...,2021-09-24
3,330614197,1-in x 6-in Western Red Cedar S1S2E Fence Board,$6.77 to $13.41,0 out of 5 starts. Read reviews.,No Reviews. Read reviews.,https://da.lowes.ca/webassets/images/333030_04...,2021-09-24
4,330614189,2-in x 6-in Western Red Cedar Lumber,$27.03 to $40.54,0 out of 5 starts. Read reviews.,No Reviews. Read reviews.,https://da.lowes.ca/webassets/images/4310_0505...,2021-09-24


#### Cleaning Lowes Columns: 

Similar to Home Depot, let's clean some of our columns to make them easier handle and extract some important information embedded within some columns. 
- Extract dimension data, create separate columns for thickness, width, and length
- Clean price, avg_rating, review_count, and product_id columns, transform to appropriate data type. 


In [15]:
# Extract dimensions info 
#Use 'get_dimensions' function defined earlier for Home Depot scrape. 

lowes_df['dimensions'] = lowes_df['description'].apply(get_dimensions)
lowes_df[['thickness','width','length']] = lowes_df['dimensions'].apply(pd.Series)


#Price - account for price ranges

lowes_df['price'] = lowes_df['price'].str.replace('$',"").str.split(" ")
lowes_df['price_high_range'] = lowes_df['price'].apply(lambda x: x[-1] if len(x)>1 else None).astype('float')
lowes_df['price_low_range'] = lowes_df['price'].apply(lambda x: x[0]).astype('float')
lowes_df['price'] = lowes_df['price_low_range']


# Avg_rating - rating is always out of 5, wanted value is always the first number.

lowes_df['avg_rating'] = lowes_df['avg_rating'].str.split(" ").apply(lambda x: x[0]).astype('float')


#Rating count - grab first number, or input 0 for case where product doesn't have reviews

def check_for_reviews (cell): 
    match = re.findall(r'\d+', cell)
    if len(match) > 0: 
        return int(match[0])
    else: return 0
    
lowes_df['review_count'] = lowes_df['review_count'].apply(check_for_reviews).astype('int')
lowes_df['sku'] = lowes_df['sku'].astype('int')
lowes_df['retailer'] = 'Lowes'
lowes_df['date'] = pd.to_datetime(lowes_df['date'])

#Rearrange column order
reorder_cols = ['sku', 'description','retailer','price', 'dimensions', 'thickness', 'width', 'length','avg_rating', 'review_count', 'date',  'image'] 
lowes_df = lowes_df[reorder_cols]

lowes_df.head(5)




,sku,description,retailer,price,dimensions,thickness,width,length,avg_rating,review_count,date,image
0,49127,2-in x 12-in x 8-ft Kiln-Dried Whitewood Lumber,Lowes,24.89,"(2.0, 12.0, 8.0)",2.0,12.0,8.0,1.00,1,2021-09-24,https://da.lowes.ca/webassets/images/49127_019...
1,28512,Canfor 2-in x 4-in x 14-ft Spruce Pine Fir Lum...,Lowes,10.99,"(2.0, 4.0, 14.0)",2.0,4.0,14.0,4.37,89,2021-09-24,https://da.lowes.ca/webassets/images/28512_089...
2,330614202,2-in x 12-in Kiln-Dried SPF Lumber,Lowes,31.49,"(2.0, 12.0, None)",2.0,12.0,NaN,1.00,1,2021-09-24,https://da.lowes.ca/webassets/images/49227_019...
3,330614197,1-in x 6-in Western Red Cedar S1S2E Fence Board,Lowes,6.77,"(1.0, 6.0, 1.0)",1.0,6.0,1.0,0.00,0,2021-09-24,https://da.lowes.ca/webassets/images/333030_04...
4,330614189,2-in x 6-in Western Red Cedar Lumber,Lowes,27.03,"(2.0, 6.0, None)",2.0,6.0,NaN,0.00,0,2021-09-24,https://da.lowes.ca/webassets/images/4310_0505...


In [16]:
lowes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   sku           15 non-null     int64         
 1   description   15 non-null     object        
 2   retailer      15 non-null     object        
 3   price         15 non-null     float64       
 4   dimensions    15 non-null     object        
 5   thickness     15 non-null     float64       
 6   width         15 non-null     float64       
 7   length        5 non-null      float64       
 8   avg_rating    15 non-null     float64       
 9   review_count  15 non-null     int64         
 10  date          15 non-null     datetime64[ns]
 11  image         15 non-null     object        
dtypes: datetime64[ns](1), float64(5), int64(2), object(4)
memory usage: 1.5+ KB


## Merging the DataFrames from each source

Now that we have two dataframes with nearly identical formats, let's merge them and output to a single csv file.

In [10]:
# Lowes data doesn't include a "url" column, but merging first on HD ensures it remains. 
# All other columns are the same
merged_df = pd.concat([hd_df, lowes_df], axis = 0, ignore_index = True)

In [11]:
merged_df.head(3)

,sku,description,retailer,price,dimensions,thickness,width,length,avg_rating,review_count,date,url,image
0,1000112108,2-inch x 4-inch x 8 ft. SPF Dimensional Lumber,Home Depot,6.58,"(2.0, 4.0, 8.0)",2.0,4.0,8.0,3.7443,884,2021-09-24 00:00:00,https://www.homedepot.ca/product/2-inch-x-4-in...,https://homedepot.scene7.com/is/image/homedepo...
1,1000100158,2x4x10 SPF Dimension Lumber,Home Depot,13.48,"(2.0, 4.0, 10.0)",2.0,4.0,10.0,3.8594,192,2021-09-24 00:00:00,https://www.homedepot.ca/product/2x4x10-spf-di...,https://homedepot.scene7.com/is/image/homedepo...
2,1000173739,HDG 1x2x8 Framing Lumber,Home Depot,3.78,"(1.0, 2.0, 8.0)",1.0,2.0,8.0,3.7595,158,2021-09-24 00:00:00,https://www.homedepot.ca/product/hdg-1x2x8-fra...,https://homedepot.scene7.com/is/image/homedepo...


In [12]:
## Export merged_df to CSV with today's date in the title. 
file_name = str(dt.date.today())+"_Dimensional_Lumber.csv"
merged_df.to_csv(path_or_buf="Output/"+file_name)

print("Data scraped & saved on {date}".format(date=dt.date.today()))


Data scraped & saved on 2021-09-24
